# RK4 Model

### Trajectory of golf ball 

In this notebook, we compare the trajectory of a golf ball computed with a model that includes drag force and Magnus effect to the the trajectory given by a HD Golf simulator. Our model uses the same initial conditions as the HD Golf simulator.

## Resources

Note that it's not unusual to see the variable $y$ used for $v_x$. This can be confusing.

- [Coupled masses](http://scipy-cookbook.readthedocs.io/items/CoupledSpringMassSystem.html). This is a clear example.
- [Projectile motion with drag](http://ipython-books.github.io/123-simulating-an-ordinary-differential-equation-with-scipy/). This uses some interesting numpy tricks like the `r_()` function, so it's harder to figure out.

In [7]:
import numpy as np #math and arrays
import matplotlib.pyplot as plt #used for graphing
import scipy.integrate as spi 
from scipy.optimize import minimize
import pandas as pd

In [8]:
# Read data file and define dataframe object (df for dataframe)
df = pd.read_table('simulator-spin-data.txt')
df.head(13)

,Shot Distance (m),Carry (m),Bounce & Roll (m),Ball Speed (m/s),Club Speed (m/s),Launch Angle (deg),Club Path,Club Face Angle (deg),Smash Factor,Horizontal Angle (deg),Apex (m),Impact Position,Spin (rpm),X Peak Point (m),Y Peak Point (m),X End Point (m),Y End Point (m)
0,113.8,111.5,2.2,40.2,33.0,27.0,-2.8,0.0,1.22,-2.8,22.4,CENTER,7881,66.0,22.4,113.0,0
1,118.6,116.1,2.5,42.6,35.7,19.4,-1.3,0.0,1.19,-1.3,17.6,TOE,9149,71.3,17.6,118.6,0
2,108.3,104.0,4.3,38.0,36.5,23.4,0.7,0.1,1.04,-0.7,16.6,N/A,7204,61.0,16.6,108.0,0
3,111.1,109.7,1.3,40.5,35.9,25.6,-0.2,0.1,1.13,-0.1,21.7,TOE,8958,66.0,21.7,111.0,0
4,106.1,103.0,3.1,37.8,34.5,25.9,-1.1,0.1,1.10,-1.2,18.5,TOE,7656,61.0,18.5,106.0,0
5,110.5,108.7,1.8,39.9,36.5,25.4,-5.4,0.1,1.09,-5.5,20.8,TOE,8544,65.0,20.8,110.0,0
6,105.5,102.8,2.7,37.9,34.5,25.3,-2.4,0.2,1.10,-2.2,18.4,N/A,7994,61.0,18.4,105.0,0
7,118.9,116.5,2.4,41.7,34.9,25.4,-3.3,0.3,1.19,-3.0,22.9,CENTER,7867,69.0,22.9,118.0,0
8,120.0,118.7,1.3,43.1,36.2,25.2,-1.7,0.3,1.19,-2.0,24.6,CENTER,8884,72.0,24.6,120.0,0
9,116.1,115.3,0.8,42.6,36.6,24.9,-2.0,0.3,1.17,-1.7,23.6,TOE,9535,70.0,23.6,116.0,0


In [9]:
rows=df['Carry (m)'].size
rows

12

In [10]:
def f(r, t, p):    
    """
    Defines the differential equations for the system.

    Arguments:
        r :  array of the state variables:
                  r = [x,vx,y,vy]
        t :  array of time values
        p :  array of the parameters:
                  p = [m,g,rho,Cd,Cm,R,A,spinrate]
    """
    m=p[0]
    g=p[1]
    rho=p[2]
    Cd=p[3]
    Cm=p[4]
    R=p[5]
    A=p[6]
    spinrate=p[7]
    
    x=r[0]
    y=r[2]
    vx=r[1]
    vy=r[3]

    # compute the derivative of velocity 
    speed=np.sqrt(vx**2+vy**2)
    vxdot = -1/2*rho*Cd*A/m*speed*vx - 1/2*Cm*rho*np.pi*R**3*spinrate*vy/m
    vydot = -g + -1/2*rho*Cd*A/m*speed*vy + 1/2*Cm*rho*np.pi*R**3*spinrate*vx/m
    
    #return [vx,vy,vxdot,vydot]
    return np.array([vx,vxdot,vy,vydot])

In [13]:
def testCoef(dragCoeffs):
    #Return RMS 
    #constants for the model
    Cd = dragCoeffs[0]
    Cm = dragCoeffs[1]
    rho = 1.225 #air density, kg/m^3
    R = 0.04267/2 #radius of ball, m
    A = np.pi*R**2 #area
    g = 9.8        #Earth's grav field strength
    m = 0.0456     #mass of the ball, kg
    spinrate = 0   #this depends on the shot and will be changed within the main loop

    #pass parameters to the ode function
    params=[m,g,rho,Cd,Cm,R,A,spinrate]
    
    #loop through all rows
    rms_total = 0
    for row in range(0,rows):    

        df_row=df.iloc[row]

        #data for HD Golf trajectory
        x_end_HD = df_row['Carry (m)']
        x_apex_HD = df_row['X Peak Point (m)']
        y_apex_HD = df_row['Y Peak Point (m)']

        #Do not need a Y_end_HD because the simulator has the golf ball ending at y=0

        x_hdgolf = [0, x_apex_HD, x_end_HD]
        y_hdgolf = [0, y_apex_HD, 0]    

        #initial conditions for the ball
        launch_angle = df_row['Launch Angle (deg)'] * np.pi/180 #radians
        launch_speed = df_row['Ball Speed (m/s)'] #m/s
        rpm = df_row['Spin (rpm)'] #rev per minute 
        spinrate = rpm /60 *2* np.pi
        params[7]=spinrate #this variable is kept in the parameters list

        # trajectory data array: [x,vx,y,vy]
        r0 = np.zeros(4) #four columns of data

        #Define initial positions and velocity
        r0[0] = 0 # x(0)
        r0[1] = launch_speed*np.cos(launch_angle) # vx(0)
        r0[2] = 0 # y(0)
        r0[3] = launch_speed*np.sin(launch_angle) # vy(0)

        #Solve the DE using RK4
        t_array = np.linspace(0., 7, 1000)
        r = spi.odeint(f, r0, t_array,args=(params,))


        #Print this line to differentiate between trials 
        #print("\n\n")

        y_apex_model=np.max(r[:,2]) #y value of peak
        apex_index, = np.where(r[:,2] == y_apex_model)
        x_apex_model = r[:,0][int(apex_index)]
        #print("model apex (x,y) = (", x_apex_model, ", ", y_apex_model, ")m")
        #print("HD apex (x,y) = (", x_apex_HD, ", ", y_apex_HD, ")m")

        y_no_initial=np.delete(r[:,2], 0) #array without the initial values
        abs_y=np.abs(y_no_initial) # |y| so negative values will be positive
        index_ground=np.argmin(abs_y) #index of array when |y| is a min (at ground)
        carry=r[:,0][index_ground+1] #x value when projectile hits the ground
        x_end_model=carry
        y_end_model=r[:,2][index_ground+1] #y value when projectile hits the ground
        #print("model end (x,y) = (", x_end_model, ", ", y_end_model, ")m")
        #print("HD end (x,y) = (", x_end_HD, ", 0)m")




        #Difference of peak point between HD and model 
        dPeak_sq = (x_apex_HD - x_apex_model)**2 + (y_apex_HD - y_apex_model)**2

        #Difference of end point between HD and model 
        dEnd_sq = (x_end_HD - x_end_model)**2 + (y_end_model)**2 

        rms = np.sqrt((dPeak_sq + dEnd_sq)/2)
        rms_total = rms_total + rms 

        #print(rms)

    #print("\n\n\n\n")        

    RMS_avg = rms_total/rows
    return RMS_avg

    #print('Average RMS:', RMS_avg)

In [14]:
bnds = ((0.1,0.9),(0.1,0.9))
res = minimize(testCoef, (0.8,0.8), bounds = bnds)
print(res.x)

[ 0.41123417  0.59101698]


In [15]:
avg_rms=testCoef(res.x)
print(avg_rms)

2.94333326845
